In [ ]:
!pip install transformers datasets evaluate > /dev/null

In [ ]:
from transformers import pipeline
from datasets import load_dataset
from evaluate import evaluator
import evaluate

In [ ]:
clf_metrics = evaluate.combine(["accuracy",
                                "f1",
                                "precision",
                                "recall"])

In [ ]:
pipe = pipeline("text-classification",
                model="lvwerra/distilbert-imdb")

In [ ]:
data = load_dataset("imdb",
                    split="test").shuffle().select(range(1000))

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
task_evaluator = evaluator("text-classification")

In [ ]:
# GPU t4 google colab
results = task_evaluator.compute(model_or_pipeline=pipe,
                                 data=data,
                                 metric=clf_metrics,
                       label_mapping={"NEGATIVE": 0, "POSITIVE": 1},)

In [ ]:
print(results)

{'accuracy': 0.937, 'f1': 0.9379310344827586, 'precision': 0.9444444444444444, 'recall': 0.9315068493150684, 'total_time_in_seconds': 310.984303651, 'samples_per_second': 3.2155963766011904, 'latency_in_seconds': 0.310984303651}


In [ ]:
# CPU google colab
results = task_evaluator.compute(model_or_pipeline=pipe,
                                 data=data,
                                 metric=clf_metrics,
                                 label_mapping={"NEGATIVE": 0, "POSITIVE": 1},)

In [ ]:
print(results)

{'accuracy': 0.929, 'f1': 0.9329556185080263, 'precision': 0.9481765834932822, 'recall': 0.9182156133828996, 'total_time_in_seconds': 526.6745037599999, 'samples_per_second': 1.8987059234135428, 'latency_in_seconds': 0.5266745037599999}


The time performances can give useful indication on model speed for inference but should be taken with a grain of salt: they include all the processing that goes on in the pipeline. This may include tokenizing, post-processing, that may be different depending on the model. Furthermore, it depends a lot on the hardware you are running the evaluation on and you may be able to improve the performance by optimizing things like the batch size.